In [1]:
### Benjamin Tollison ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy.solvers.pde import pdsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate, Function, Eq
import numpy as np
import matplotlib.pyplot as plt
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [2]:
rho = 1.2250554566 # kg/m
alpha,delta_E,theta = symbols('\\alpha \\delta_E \\theta')
gamma = theta - alpha
W = 20500*0.453592 # kg
S_W = 300*0.3048**2 # m^2
c_W = 11.32*0.3048 # m
v = 500 * 0.3048 # m/s
q_infty = 0.5*rho*v**2
C_L = 0.14 + 4.2*alpha - 4.8*alpha**2 + 0.44*delta_E
C_M = -0.02 + 0.04*alpha - 0.6*delta_E - 0.08*alpha*delta_E + .08*delta_E**2 + 0.5*alpha*delta_E
eqn1 = q_infty*S_W*C_L - W
eqn2 = q_infty*S_W*c_W*C_M
dof = sp.nonlinsolve([eqn1,eqn2],[alpha,delta_E])
alpha_0, delta_E0 = next(iter(dof))
alpha_0_symbol,delta_E0_symbol = symbols('\\alpha_0 \\delta_{E_0}')
displayEquations(alpha_0_symbol,alpha_0)
displayEquations(delta_E0_symbol,delta_E0)

<IPython.core.display.Math object>

<IPython.core.display.Math object>